In [1]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import torchvision
from torchvision.transforms import v2
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import train_test_split

eurosatNTData = torchvision.datasets.EuroSAT(root = './data', download = True, transform = None)
device = "cuda" if torch.cuda.is_available() else "cpu"
useAugmentation = False


100%|██████████| 94.3M/94.3M [00:00<00:00, 359MB/s]


In [33]:
def GetLoaders(isAugmented = False, batchSize = 32):

  indices = list(range(len(eurosatNTData)))
  labels = eurosatNTData.targets

  trainIdx, testIdx = train_test_split(
      indices,
      test_size=0.1,
      random_state=42,
      stratify=labels
  )

  noAUGTransform = v2.Compose([
      v2.ToTensor(),
      v2.Normalize(mean = [0.5], std = [0.5]),
  ])

  AUGTransform = v2.Compose([
    v2.RandomPerspective(distortion_scale=0.2, p=0.5),
    v2.RandomAdjustSharpness(sharpness_factor=2.0, p=0.5),
    v2.RandomAutocontrast(p=0.5),
    v2.RandomApply([v2.ColorJitter(brightness=0.5)],p=0.4) ,
    v2.RandomApply([v2.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5.0))],p=0.8),
    v2.ToTensor(),
    v2.Normalize(mean = [0.5], std = [0.5])
])


  if(isAugmented):
    notAugmentedTrain = Subset(torchvision.datasets.EuroSAT(root="./data", transform=noAUGTransform),trainIdx)
    augmentedTrain = Subset(torchvision.datasets.EuroSAT(root="./data", transform=AUGTransform),trainIdx)
    trainDataset = torch.utils.data.ConcatDataset([augmentedTrain,notAugmentedTrain])
  else:
    trainDataset = Subset(torchvision.datasets.EuroSAT(root="./data", transform=noAUGTransform),trainIdx)

  testDataset = Subset(torchvision.datasets.EuroSAT(root="./data", transform=noAUGTransform),testIdx)
  trainLoader = torch.utils.data.DataLoader(trainDataset, batch_size = batchSize, shuffle = True, num_workers = 2, pin_memory=True)
  testLoader = torch.utils.data.DataLoader(testDataset, batch_size = batchSize, shuffle = False)
  print(len(trainDataset))
  return trainLoader, testLoader


In [34]:
trainLoader, testLoader = GetLoaders(useAugmentation, 64)

48600


/usr/local/lib/python3.12/dist-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [35]:
class CNN(nn.Module):
  def __init__(self, inChannels, numClasses):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(in_channels = inChannels, out_channels= 16, kernel_size = 3, stride = 1, padding = 1)
    self.pool = nn.MaxPool2d(kernel_size = 2, stride = 2)
    self.conv2 = nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 3, stride = 1, padding = 1)
    self.fc1 = nn.Linear(32 * 16 * 16, 128)
    self.fc2 = nn.Linear(128, numClasses)
    self.dropfc1 = nn.Dropout(p=0.2)
    # self.dropconv2 = nn.Dropout2d(p=0.1)

  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = self.pool(x)
    x = F.relu(self.conv2(x))
    x = self.pool(x)
    x = x.view(-1, 32 * 16 * 16)
    x = F.relu(self.dropfc1(self.fc1(x)))
    x = self.fc2(x)
    return x

model = CNN(inChannels = 3, numClasses = 10).to(device)

In [36]:
#Hyper parameters
learningRate = 1e-3
epochs = 80
lossFunction = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learningRate)
scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max=epochs)
losses = []

In [41]:
def train():
  for epoch in range(epochs):
    epochLoss = 0
    numBatches = 0
    for X, y in trainLoader:
      X = X.to(device, non_blocking=True)
      y = y.to(device, non_blocking=True)
      optimizer.zero_grad()
      pred = model(X)
      loss = lossFunction(pred, y)
      loss.backward()
      optimizer.step()
      epochLoss += loss.item()
      numBatches += 1
    scheduler.step()
    avgLoss = epochLoss/numBatches
    losses.append(avgLoss)
    print(f'Epoch {epoch+1}, Loss: {avgLoss:.9f}, Lr: {optimizer.param_groups[0]["lr"]:.6}')

train()

Epoch 1, Loss: 0.192450387, Lr: 0.0
Epoch 2, Loss: 0.195868870, Lr: 3.85482e-07
Epoch 3, Loss: 0.194503068, Lr: 1.54133e-06
Epoch 4, Loss: 0.193581113, Lr: 3.46577e-06
Epoch 5, Loss: 0.194837878, Lr: 6.15583e-06
Epoch 6, Loss: 0.192171742, Lr: 9.60736e-06


KeyboardInterrupt: 

In [42]:
def check_accuracy():


    num_correct = 0
    num_samples = 0
    model.eval()  # Set the model to evaluation mode

    with torch.no_grad():  # Disable gradient calculation
        for x, y in testLoader:
            x = x.to(device,non_blocking=True)
            y = y.to(device,non_blocking=True)
            # Forward pass: compute the model output
            scores = model(x)
            _, predictions = scores.max(1)  # Get the index of the max log-probability
            num_correct += (predictions == y).sum()  # Count correct predictions
            num_samples += predictions.size(0)  # Count total samples

        # Calculate accuracy
        accuracy = float(num_correct) / float(num_samples) * 100
        print(f"Got {num_correct}/{num_samples} with accuracy {accuracy:.2f}%")

    model.train()  # Set the model back to training mode

# Final accuracy check on training and test sets
check_accuracy()


Got 2327/2700 with accuracy 86.19%
